In [1]:
import os
import pandas as pd

fsaverage_parcel_file = 'data/ds001226-fmriprep/sourcedata/freesurfer/fsaverage/mri/aparc+aseg.mgz'
derivative_dir = 'data/derivative/'
tumor_masks_dir = 'data/ds001226/derivatives/tumor_masks/'
lut_table_file = 'data/derivative/parcellation_lut.txt'

For each subject we need to compute which regions on the Desikan-Killiany parcellation have tumor (this is, at least one voxel intersection). This is performed as following:
- Mask the fsaverage with the tumor mask
- Compute stats using Desikan-Killiany lut table
- Read stats and output for each region tumor volume (or 0.0 if none)

Notes:
- Freesurfer Desikan-Killiany has 35+35 regions in its LUT (ex. "colortable_desikan_killiany.txt") and not 34+34, why?
    - One region is ignored based on datasheet delivered from dataset authors: ctx-XX-corpuscallosum

In [2]:
for pat in os.listdir(tumor_masks_dir):
    if os.path.isdir(tumor_masks_dir + pat):
        print ("------------- Processing patient " + pat + " -------------")

        pat_tumor_mask_file = os.path.join(tumor_masks_dir, pat, 'anat', 'sub-' + pat.replace("sub-", "") + '_space_MNI_label-tumor.nii')
        output_pat_dir = os.path.join(derivative_dir, pat)

        # Create output dir if non existent
        os.makedirs(output_pat_dir, exist_ok=True)

        # We need to mask fsaverage with tumor mask
        # $ mri_mask volume mask output
        output_masked_vol_file = os.path.join(output_pat_dir, 'aparc+aseg-tumor_masked.mgz')
        os.system('mri_mask ' + fsaverage_parcel_file + ' ' + pat_tumor_mask_file + ' ' + output_masked_vol_file)

        # Output masked file stats
        output_raw_maked_stats_file = os.path.join(output_pat_dir, 'aparc-aseg-tumor_masked.stats')
        os.system('mri_segstats --seg ' + output_masked_vol_file + ' --ctab ' + lut_table_file + ' --sum ' + output_raw_maked_stats_file)

        # Read the created stats file
        segstats_table = pd.read_csv(
            output_raw_maked_stats_file,
            sep='\s+',
            comment='#',
            header=None,
            names=['Index', 'SegId', 'NVoxels', 'Volume_mm3', 'StructName'],
            index_col='StructName'
        )

        # Read lut table so we know region indices
        lut_table = pd.read_csv(
            lut_table_file,
            sep='\s+',
            comment='#',
            header=None,
            names=['No.', 'StructName', 'R', 'G', 'B', 'A'],
            # index_col='StructName'
        )

        # Let's create a table that has for each region of the parcellation its volumetric tumor intersection in mm³
        tumor_regions = lut_table[['StructName']].copy()
        tumor_regions['Volume_mm3'] = {k: segstats_table['Volume_mm3'].get(r['StructName'], 0.0) for k, r in lut_table.iterrows()}

        # Save tumor regions
        output_tumor_regions_file = os.path.join(output_pat_dir, 'tumor_regions.csv')
        tumor_regions.to_csv(output_tumor_regions_file, sep='\t', index=True, encoding='utf-8')



------------- Processing patient sub-PAT01 -------------
DoAbs = 0
maskval=0, outval=0
INFO: MRImask() using MRImaskDifferentGeometry()
Writing masked volume to data/derivative/sub-PAT01/aparc+aseg-tumor_masked.mgz...done.

7.4.0
cwd 
cmdline mri_segstats --seg data/derivative/sub-PAT01/aparc+aseg-tumor_masked.mgz --ctab data/derivative/parcellation_lut.txt --sum data/derivative/sub-PAT01/aparc-aseg-tumor_masked.stats 
sysname  Linux
hostname junky-udg-labtop
machine  x86_64
user     junky
whitesurfname  white
UseRobust  0
Loading data/derivative/sub-PAT01/aparc+aseg-tumor_masked.mgz
Voxel Volume is 1 mm^3
Generating list of segmentation ids
Found  68 segmentations
Computing statistics for each segmentation

Reporting on   3 segmentations
Using PrintSegStat
mri_segstats done
------------- Processing patient sub-PAT02 -------------
DoAbs = 0
maskval=0, outval=0
INFO: MRImask() using MRImaskDifferentGeometry()
Writing masked volume to data/derivative/sub-PAT02/aparc+aseg-tumor_masked.mgz